# ニューラルネットワーク

ここでは，ディープラーニングのベースとなっているニューラルネットワーク (Neural Network) について紹介していきます．このニューラルネットワークをベースに画像向けの Convolutional Neural Network (CNN) や系列向けの Recurrent Neural Network (RNN) のように実問題における表現まで学習できるような工夫を追加したものを総称してディープラーニングと呼び，後半で扱っていきます．ディープラーニングの理論も魅力的ですが，まずはニューラルネットワークの数学をしっかりと理解しておくことが重要です．ニューラルネットワークの数学の理解には，重回帰分析の理解が大いに役立つはずです．またこれに加えて，非線形なモデリングや最適化の理論が新たに加わるため，楽しんで学んでいきましょう．ニューラルネットワークでもこれまでと同様に，モデルを決め，目的関数を決め，パラメータの更新を行います．

## Step1. モデルを決める

まずはニューラルネットワークの構造を数式ではなくグラフィカルに見てみましょう．

![](images/3/01.png)

ひとつひとつの丸い部分のことを**ノード**と呼び，そのノードの集まりを**層**と呼びます．そして，一番初めの層を**入力層**，最後の層を**出力層**，そしてその間を**中間層**もしくは**隠れ層**と呼びます．このモデルは入力層，中間層，出力層の３層の構造となっていますが，中間層の数を増やすことでさらに多層のニューラルネットワークとして定義することもできます．こちらでは各層間の全てのノードが結合されているため，**全結合のニューラルネットワーク**とも呼び，ニューラルネットワークの最も基礎的な構造です．

入力変数の扱い方はこれまでと同様ですが，出力変数の扱い方がこれまでと異なります．例えば，上図では白ワインと赤ワインを分類するような問題を例に挙げていますが，SVMなどの機械学習の場合，$y \in \{-1, 1\}$ のようにひとつの出力変数のとる値で属するカテゴリを指定していました．それに対して，ニューラルネットワークでは，白ワイン用の出力変数 $y_{1}$ と赤ワイン用の出力変数 $y_{2}$ のカテゴリの数だけ出力の変数が準備されています．なぜこのような構造となっているのでしょうか．

![](images/3/02.png)

この答えの前に，最終的にニューラルネットワークではどのような出力の結果が得られるかを見てみましょう．例えば，年数が3年物でアルコール度数が14度，色合いが0.2，匂いが0.8のように定量評価できるワインがあるとしましょう．内部の計算は後述するとして，このようなデータをニューラルネットワークに与えると，白ワイン $y_{1} = 0.15$, 赤ワイン $y_{2}= 0.85$ という値が得られるとしましょう．そして，最終的に最も大きな値である赤ワインを分類の結果として採用します．ここで出力の値に注目すると，合計すると1になっており，これは偶然ではなく一般的に成立するような機構になっています．

いま注目して欲しい点としては，SVMのような分類の手法では，白ワインか赤ワインかといったどのカテゴリに属するかしかわからなかったものの，ニューラルネットワークでは，どのクラスにどのくらいの確率で属すかまで知ることができます．前者の各カテゴリの境界線だけを求める手法を**識別関数**，後者の各カテゴリに属する確率まで求める手法を**識別モデル**と呼びます．また，これと並んで確率の知見を取り入れてデータの分布を直接求める手法を**生成モデル**と呼びますが，これは**ベイズ統計**をベースとしているため，さらに余裕が出てきてから取り組むことをお勧めします．

そして，もう一点確認したいこととして，ニューラルネットワークは回帰か分類どちらを扱う手法であるかです．答えは両方扱うことができます．ただし，最終的な出力は連続値となっていることがわかると思います．そのため，ニューラルネットワークは基本的には回帰の手法であり，この出力の値を使って分類も行えるということで，回帰も分類も両方の問題設定が扱えるわけです．

それでは，ここからニューラルネットワークの数学を本格的に扱っていきましょう．ニューラルネットワークでは数式もプログラミングも**2層で1セット**として扱うことを覚えておいてください．多いものだと100層を超えるため全体のモデルで考えると複雑になりすぎるため，2層ごとに分割してその連結で考えていきます．線形変換と非線形変換の2つの手順を流れに沿って紹介していきます．

### 線形変換

![](images/3/03.png)

線形変換では，重回帰分析のときと同様に，重み ($w$) ×入力 ($h$) + バイアス ($b$) の計算を行います．これからは，$h$ が文字としてよく登場しますが，隠れ層 (hidden layer) の頭文字である $h$ です．全てのノードについてそれぞれ考えると，

$$
\begin{aligned}
u_{11}&=w_{11}h_{01}+w_{12}h_{02}+w_{13}h_{03}+w_{14}h_{04}+b_{1} \\
u_{12}&=w_{21}h_{01}+w_{22}h_{02}+w_{23}h_{03}+w_{24}h_{04}+b_{2} \\
u_{13}&=w_{31}h_{01}+w_{32}h_{02}+w_{33}h_{03}+w_{34}h_{04}+b_{2}
\end{aligned}
$$

のように計算を行います．これを行列でまとめると，

$$
\begin{aligned}
\begin{bmatrix}
u_{11} \\
u_{12} \\
u_{13}
\end{bmatrix}&=\begin{bmatrix}
w_{11} & w_{12} & w_{13} & w_{14} \\
w_{21} & w_{22} & w_{23} & w_{24} \\
w_{31} & w_{32} & w_{33} & w_{34}
\end{bmatrix}\begin{bmatrix}
h_{01} \\
h_{02} \\
h_{03} \\
h_{04}
\end{bmatrix}+\begin{bmatrix}
b_{1} \\
b_{2} \\
b_{3}
\end{bmatrix}\\
\boldsymbol{u}_{1}&=\boldsymbol{W}\boldsymbol{h}_{0}+\boldsymbol{b}
\end{aligned}
$$

のようにまとめることができます．本来は $\boldsymbol{W}$ や $\boldsymbol{b}$ にも添え字をつけるべきではありますが，記号が複雑になってくるため今回は省略します．厳密な定義としては $\boldsymbol{u} = \boldsymbol{W}\boldsymbol{h}$ が線形変換であるが，重回帰分析でも紹介した通り，バイアス $\boldsymbol{b}$ を $\boldsymbol{W}$ に包含すればこのような式でも表現できるため，この $\boldsymbol{u} = \boldsymbol{W}\boldsymbol{h} + \boldsymbol{b}$ の計算でも線形変換と呼んでいます．

### 非線形変換

![](images/3/04.png)

これまでは，上図左側のように $y = wx + b$ で扱える範囲の手法のみ紹介していましたが，実データでは上図右のように入出力間の関係が直線（や超平面）では表現できない非線形な関係性の場合もあります．この場合，どのように対処すれば良いでしょうか．すでに紹介していたサポートベクターマシンでは**カーネルトリック**と呼ばれる方法を使いますが，数式を理解するのが一苦労です．ニューラルネットワークではもう少し考え方が簡単であり，内部に非線形な関数を各ノードが取り入れます．この組み合わせをうまく調整していくことにより，入出力間の非線形性に対応できると考えました．この非線形変換を行う関数を **活性化関数** と呼びます．

![](images/3/05.png)

ニューラルネットワークでは上図に示す**シグモイド関数**
$$
h = f(u) = \dfrac{1}{1+e^{-u}}
$$
がよく用いられてきて，教科書でも良く紹介されています．しかし，最近ではあまり使われないものとなってきており，理由は後述しますが**勾配が消失する**といわれる問題であり，多層のニューラルネットワークにする際のボトルネックになってしまいました．そこで，最近では，よく登場するものとして，**Relu関数**

![](images/3/06.png)


$$
h = f(u) = \max(0, u)
$$
が使われることが多くなりました．この関数は入力が負の値の場合に出力が0で，正の値はそのまま出力するといった特性を持っています．このRelu関数はシグモイド関数で問題となっていた勾配が消失する問題に対する解決策のひとつであり，多層のニューラルネットワークを構成する際になくてはならない存在となりました．

### 数値例で計算の流れを確認

![](images/3/07.png)

ここでは，これまでに数式が多くイメージが湧きにくかったため，上図の数値例で出力 $y$ までのニューラルネットワークの計算の流れを確認していき，具体的な計算手順のイメージを把握しておきましょう．今回は計算を簡略化するためバイアス $\boldsymbol{b}$ は省略しています．数値例として，$\boldsymbol{x}^T = \begin{bmatrix} 2 & 3 & 1 \end{bmatrix}$ が与えられた時の出力 $y$ を計算してみましょう．

機械学習のアルゴリズムを考える場合に，肝となることが「どこから計算を始めるか」です．パラメータ $\boldsymbol{W}$ が計算の過程で登場するのですが，この値が決まっていないと計算を始めることができません．重回帰分析では，定式化したまま微分の値が0となるように計算ができましたが，ニューラルネットワークではモデルの構造上，この微分の値が0となるようなパラメータを直接求めることは難しいという背景があります．その代わり，微分を利用した別の方法でうまくパラメータを最適化していけるため，その方法は後述します．そして，パラメータを直接求めることができない場合は，ひとまずランダムでも良いので，パラメータの初期値を仮で決めます．そこからどの程度悪かったのか・良かったのかを見て，良い方向に進むようにさらにパラメータを調整していくといったアプローチをとります．

パラメータの初期値に対して，線形変換を計算すると，
$$
\begin{aligned}
u_{11}&=3\times 2+1\times 3+2\times 1=11\\
u_{12}&=-2\times 2-3\times 3-1\times 1=-11
\end{aligned}
$$

となります．次に非線形変換ですが，活性化関数としてRelu関数を採用した場合，
$$
\begin{aligned}
h_{11} &= \max(0, 11) = 11 \\
h_{12} &= \max(0, -11)  = 0
\end{aligned}
$$

となります．そして，$y$ まで同様に計算すると，
$$
y = 3 \times 11 + 2 \times 0 = 33
$$

となります．理論の話の際には難しく感じたかもしれませんが，実際の計算は足し算と掛け算が主であり，とても簡単であることがわかります．

さて，次項以降では残していたパラメータ更新のための原理について数式を使ってさらに理解を深めていきましょう．

## Step2. 目的関数を決める

ニューラルネットワークでは，回帰と分類の両方に対応することができますが，問題設定によって目的関数は異なります．回帰では重回帰分析と同様に**二乗誤差**を使用します．Chainerのフレームワークで実装されているものは**平均二乗誤差** (mean squared error)

$$
\mathcal{L} = \dfrac{1}{N} \sum_{n=1}^{N}(t_{n} - y_{n})^{2}
$$
であり，重回帰分析の総和に対して平均を採用しているだけであり，これまでの議論と本質的な部分が変わることはありません．つぎに，分類では**クロスエントロピー**

$$
\mathcal{L} = - \dfrac{1}{N}\sum_{n=1}^{N}t_{n}\log y_{n}
$$

を目的関数として採用します．$log$ が登場したりと難しそうに見えるかもしれませんが，この意味をさらに知りたい方は**情報理論**の基礎的な部分を学ぶと理解することができます．ひとまず，分類における損失を定量評価できる関数だと考えて進めていくことにしましょう．

それでは先ほどの例題に対して，教師データ $t = 20$ の場合の計算を行ってみると，回帰の目的関数となる平均二乗誤差は
$$
\mathcal{L} = \dfrac{1}{1} (20 - 33)^2 = 169
$$

となります．



## Step3. パラメータを最適化する

それでは，重回帰分析とは異なる点であるニューラルネットワークにおけるパラメータの最適化について考え方を理解していきましょう．まず確認として，ニューラルネットワークにおけるパラメータは，各層における $\boldsymbol{W}$ と $\boldsymbol{b}$ です．式で書かないと少しわかりにくいのですが，ニューラルネットワークでは各パラメータに対する微分を定式化したまま求めることは難しいのですが，各パラメータの値を決めた状態では各点での導関数の値は求めることができます．この各点での導関数の値は**微分係数**と呼びますが，この値がわかるということはパラメータを最適化するときには大きなメリットがあります．

それでは，図も使って微分係数を使ったパラメータの最適化について考えていきましょう．まず何も計算していない状況ですが，例えばこの点線のようなパラメータ $w$ に対する目的関数 $\mathcal{L}$ が存在するとしましょう．この例では二次関数になっていますが，実際にはもっと複雑な関数であることが多いです．実際には目的関数の形を把握することができないため，点線で示されている目的関数は「神のみぞ知る」といったものです．さて，この全体像も全く見えない中，どうやって計算機を使って最適なパラメータを見つけていくのでしょうか．

![](images/3/13.png)

ニューラルネットワークでは，はじめにランダムでも良いのでパラメータの初期値を決定すると数値例で紹介しました．例えば，ランダムに決定した結果 $w=3$ になったとしましょう．そうなると，後述する計算式で接線の傾きである微分係数が求まります．

![](images/3/10.png)

例えば，求めた結果，微分係数が 3 であったとしましょう．もちろんランダムに決めた初期値であるため，神のみぞ知る目的関数としては最適なパラメータとして決定できているわけではありません．それでは，神のみぞしる目的関数の全体像が今見えているみなさんであれば，つぎに移動するとすれば「＋方向」か「ー方向」のどちらを選ぶでしょうか．「ー方向」を選ぶ方が多いはずです．それでは，この処理を計算機で行わせるために，その戦略を考えると，傾きが正（＋）の値であったため，逆に負（ー）の方向に進むという方法はいかがでしょうか．さらに傾きの値も使って，「-1 × 3 進む」という風に考えてみます．ただし，「-1 × 3 進む」と逆に行き過ぎてしまうかもしれないため，「-1 × 0.5 × 3 進む」という風に進み具合を調整できる**学習係数** (learning rate) を導入すると，「-1.5 進む」となり，次のパラメータの値へ移動します．

![](images/3/11.png)

そして，この点においても，傾きを求めてみると，例えば -1 であったとしましょう．右肩下がりであるため，負の値となります．そうすれば，傾きが負の値の時は，逆に正（＋）の方向に移動すればよさそうであるため，「-1 × 0.5 × -1 進む」としましょう．

![](images/3/12.png)



このように「-1 × 学習係数× 微分係数」と移動していきながら，傾きが0となれば終了とすれば，全体像が見えない中でも目的関数が最小となる点に到達することができます．厳密には，初期値に最も近い点での谷である**局所最適解**に落ちる方法であり，下図に示すように常に**大域的最適解**が求まるわけではありません．初期値の選び方に大きく依存するのですが，その初期値をうまく選ぶことは難しい問題であるため，基本的には局所最適解で良しとしています．複数の初期値で学習を行い，最終的にもっとも良かったものを選択するという方法もありますが，ニューラルネットワーク以降のディープラーニングでは学習に大きく時間がかかるため，この方法も現実的ではない場合が多いといえます．

微分係数を使って上述の手順で最適化を行っていく手法を**最急降下法**と呼びます．最適化の数学で基礎として出てくる有名な方法です．また，ニューラルネットワークも含めディープラーニングでは，すべてのサンプルに対する目的関数の計算ではなく，**ミニバッチ**と呼ばれるサンプルをランダムに複数のかたまりに分割した単位で最適化を行っていくことが多く，この場合には，**確率的勾配降下法** (SGD: Stocastic Gradient Descent) と呼ばれており，Chainerの中では `SGD` としてよく登場するため覚えておきましょう．

![](images/3/14.png)



さて，本題に戻ります．先ほどの最急降下法や確率的勾配降下法を利用するためには，微分係数が求まることが前提でした．そこで，数値例の問題設定に対して，実際に微分係数を求めていきましょう．文字の取り扱いを簡単かするため，下記のように $w_{1}$ と $w_{2}$ の２つの場合について紹介します．この2つを選択する理由としては，各層ごとに計算が異なってくるためです．

![](images/3/08.png)

それでは，$w_{1}$ での微分から考えましょう．はじめに，目的関数からターゲットとなっている $w_{1}$ までの計算を洗い出すと，
$$
\begin{aligned}
\mathcal{L} &= \dfrac{1}{N}\sum_{n=1}^{N}(t_n - y_{n})^{2} \\
y_{n} &= w_{1}h_{11} + \cdots
\end{aligned}
$$

のようになります．ポイントとして，$w_{1}$ に関係しない項は $w_{1}$ で偏微分すると0になるため，「$\cdots$」のように書いています．微分の合成関数の時にも紹介しましたが，
$$
\dfrac {\partial \mathcal{L}}{\partial w_{1}}=\dfrac {\partial y_{n}}{\partial w_{1}}\dfrac {\partial \mathcal{L}}{\partial y_{n}}
$$
のように関連する項目に分解していく方法を**チェインルール**と呼びます．そして，各項目の値を求めると，
$$
\begin{aligned}
\dfrac {\partial y_{n}}{\partial w_{1}}&=h_{11} = 11 \\
\dfrac {\partial \mathcal{L}}{\partial y_{n}}&=\dfrac {1}{N}\sum ^{N}_{n=1}2\left( t_{n}-y_{n}\right) \times \left( -1\right) \\
&=-\dfrac {2}{N}\sum_{n=1}^{N}\left( t_{n}-y_{n}\right) \\
&= -2 \times (20 - 33) = 26
\end{aligned}
$$
となり，最終的な微分係数は
$$
\dfrac {\partial \mathcal{L}}{\partial w_{1}}=\dfrac {\partial y_{n}}{\partial w_{1}}\dfrac {\partial \mathcal{L}}{\partial y_{n}} = 11 \times 26 =286
$$
となりました．

次に，$w_{2}$ についても同様に考えていきましょう．目的関数からターゲットとなっている $w_{2}$ までの計算を洗い出すと，
$$
\begin{aligned}
\mathcal{L} &= \dfrac{1}{N}\sum_{n=1}^{N}(t_n - y_{n})^{2} \\
y_{n} &= w_{1}h_{11} + \cdots \\
&= w_{1} f(u_{11})  + \cdots \\
u_{11}&=w_{2}x_{1} + \cdots
\end{aligned}
$$
となります．この考え方には慣れが必要であるため，ひとつひとつ順を追っていく練習を行いましょう．これをもとにチェインルールを活用しながら $w_{2}$ での微分係数を求めていくと，
$$
\begin{aligned}
\dfrac {\partial \mathcal{L}}{\partial w_{2}}&=\dfrac {\partial y_{n}}{\partial w_{2}}\dfrac {\partial L}{\partial y_{n}}\\
&=\dfrac {\partial u_{11}}{\partial w_{2}}\dfrac {\partial y_{n}}{\partial u_{11}}\dfrac {\partial \mathcal{L}}{\partial y_{n}}\\
\dfrac {\partial u_{11}}{\partial w_2}&=x_{1}=2\\
\dfrac {\partial y_{n}}{\partial u_{11}}&=w_{1}f'\left( u_{11}\right) \\
\dfrac {\partial \mathcal{L}}{\partial y_{n}}&=-\dfrac {2}{N}\sum ^{N}_{n=1}\left( t_{n}-y_{n}\right) =26
\end{aligned}
$$

となります．ここで，$f'(u_{11})$ のところで，活性化関数であるRelu関数の微分が必要となり，Relu関数の微分は
$$
f'\left( u\right) =
\begin{cases}
1 \quad u >0\\
0 \quad u\leq 0\end{cases}
$$

のように入力される $u$ の値の符号によって変わっています．これが全体として微分を定式化できない理由であり，値によって場合分けというものが扱いにくいのですが，そのパラメータの1点における微分係数であれば問題なく，
$$
\begin{aligned}
\dfrac {\partial y_{n}}{\partial u_{11}}&=w_{1}f'\left( u_{11}\right) =3\times 1=3\\
\end{aligned}
$$
のように計算できます最終的に，
$$
\begin{aligned}
\dfrac {\partial \mathcal{L}}{\partial w_{2}}&=\dfrac {\partial u_{11}}{\partial w_{2}}\dfrac {\partial y_{n}}{\partial u_{11}}\dfrac {\partial \mathcal{L}}{\partial y_{n}}\\
&= 2 \times 3 \times 26 = 156
\end{aligned}
$$

と求まります．たとえば，学習係数 $\rho=0.01$ として最急降下法によりパラメータを更新すると，更新後の $w_{1}^{(1)}$ は更新前の $w_{1}^{(0)}$ を使って，
$$
\begin{aligned}
w_{1}^{(1)} &= w_{1}^{(0)} - \rho\dfrac{\partial \mathcal{L}}{\partial w_{1}} \\
&= 3 - 0.01 \times 286 \\
&= 0.14
\end{aligned}
$$

となります．また，更新後の $w_{2}^{(1)}$ は更新前の $w_{2}^{(0)}$ を使って，
$$
\begin{aligned}
w_{2}^{(1)} &= w_{2}^{(0)} - \rho\dfrac{\partial \mathcal{L}}{\partial w_{2}} \\
&= 3 - 0.01 \times 156 \\
&= 1.44
\end{aligned}
$$
となります．このように，パラメータの更新を行い，また，更新後のパラメータで予測値の計算および目的関数の計算を行い（**順伝播**），微分係数を求めてパラメータの更新（**逆伝播**）と繰り返していきます．重回帰分析では最初から最適なパラメータが求まっていましたが，ニューラルネットワークではパラメータの更新を行うために，順伝播と逆伝播を何度も繰り返すことが特徴的です．逆伝播における微分係数を求める効率的な方法は**誤差逆伝播法**として体系化されていますが，まずは前述した逆伝播の計算をしっかりと理解で充分であるため，余裕が出てきたタイミングで参考書を見ながら勉強してみてください．

## 勾配が消失する問題

ニューラルネットワーク最後のパートとして，活性化関数のパートで，標準的であった活性化関数であるシグモイド関数が使われなくなっていると紹介していましたが，その理由について考えていきます．関数の形を見ると，直感的にはシグモイド関数の方が対応力が高そうで，Relu関数は退屈な関数に見えます．パラメータ更新のところで実際に計算したところ，$w_{2}$ の微分を計算する際に
$$
\dfrac {\partial y_{n}}{\partial u_{11}}=w_{1}f'\left( u_{11}\right) 
$$

の計算が登場し，特に $f'(u_{11})$ のように活性化関数に関する微分が登場していることがわかると思います．

まずRelu関数と比較するために，シグモイド関数の微分を計算してみると，
$$
\begin{aligned}
f\left( u\right) &=\dfrac {1}{1+e^{-u}}=\left( 1+e^{-u}\right) ^{-1}\\
f'\left( u\right) &=-1\times \left( 1+e^{-u}\right) ^{-2}\times \left( -e^{-u}\right) \\
&=\dfrac {e^{-u}}{\left( 1+e^{-u}\right) ^{2}}\\
&=\dfrac {1+e^{-u}-1}{\left( 1+e^{-u}\right) ^{2}}\\
&=\dfrac {1+e^{-u}}{\left( 1+e^{-u}\right) ^{2}}-\dfrac {1}{\left( 1+e^{-u}\right) ^{2}} \\
&=\dfrac {1}{1+e^{-u}}-\left( \dfrac {1}{1+e^{-u}}\right) ^{2}\\
&=f\left( u\right) -\left( f\left( u\right) \right) ^{2}\\
&=f\left( u\right) \left( 1-f\left( u\right) \right)
\end{aligned}
$$

となります．ここで，微分の公式として，
$$
\begin{aligned}
(x^{-1}) ' &= -x^{-2} \\
(e^{ax})' &= ae^{ax} 
\end{aligned}
$$

の 2 つも利用しており，合成関数も使っています．そして，このシグモイド関数を微分した関数のグラフは下記のようになります．

![](images/3/09.png)

このように，シグモイド関数の微分を計算してみると，**シグモイド関数の微分係数の最大値が0.25**であることがわかります．それに対し，前述していたRelu関数の微分係数の最大値は1でした．

これにどのような差があるかというと，各パラメータの微分係数を求めた際に，シグモイド関数の場合，大きくても0.25倍となり，微分係数（各パラメータの微分係数をまとめて**勾配**ベクトルと呼びます）の値が小さくなることがわかります．今回は3層であったため，気にならない程度ですが，もし4層の場合，一番後ろの層のパラメータの微分係数を求める場合，最低でも $0.25 \times 0.25 = 0.0625$ となり，さらに勾配の値が小さくなります．つまり，ディープラーニングのように多層のニューラルネットワークの場合にシグモイド関数を使用すると，後ろの層に行くほど勾配情報が極端に小さくなっていき，最急降下法や確率的勾配降下法のような方法では，後ろの層ではパラメータの更新が実質行えないような状況となってしまいます．これが**勾配が消失する**という問題の正体です．そして，シグモイド関数に対し，Relu関数では微分係数の最大値が1であるため，層の数が増えても勾配の値が小さくなっていくことがなく，勾配が消失する問題に対処することができるわけです．ただし，シグモイド関数のような形状の活性化関数を使用したい状況もあり，その時には，形が近しく勾配の消失に関する改善がなされている**ハイパボリックタンジェント**を使用するため，こちらも余裕がでれば関数の式と微分の計算，微分係数の最大値をおさえておきましょう．